In [4]:
%matplotlib inline

import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

import QSTK.qstkutil.qsdateutil as du
import QSTK.qstkutil.tsutil as tsu
import QSTK.qstkutil.DataAccess as da

In [5]:
def gen_portfolio_allocations(symbols, resolution):
    alloc_values = np.arange(0., 1.+resolution, resolution)
    alloc_combinations = list(itertools.product(alloc_values, repeat=len(symbols)))
    alloc_combinations = pd.DataFrame(alloc_combinations, columns=symbols)
    return alloc_combinations[alloc_combinations.sum(1) == 1]

In [6]:
gen_portfolio_allocations(['a', 'b'], 0.1)

,a,b
10,0.0,1.0
20,0.1,0.9
30,0.2,0.8
40,0.3,0.7
50,0.4,0.6
60,0.5,0.5
70,0.6,0.4
80,0.7,0.3
90,0.8,0.2
100,0.9,0.1


In [7]:
def simulate_portfolio(start_date, end_date, symbols, allocation):
    timeofday = dt.timedelta(hours=16)
    timestamps = du.getNYSEdays(start_date, end_date, timeofday)
    
    dataobject = da.DataAccess('Yahoo')
    keys = ['close']
    data = dataobject.get_data(timestamps, symbols, keys)
    data_frame = dict(zip(keys, data))
    close_prices = data_frame['close'].values
    normalized_close_prices = close_prices / close_prices[0, :]
    
    daily_returns = normalized_close_prices.copy()
    tsu.returnize0(daily_returns)
    
    portfolio_std_deviation = daily_returns.std(0)
    return portfolio_std_deviation

In [8]:
simulate_portfolio(dt.datetime(2011, 1, 1), dt.datetime(2011, 12, 31), ['AAPL', 'GLD', 'GOOG', 'XOM'], [0.4, 0.4, 0.0, 0.2])

array([ 0.01645442,  0.01287063,  0.0185095 ,  0.01589745])